# CPRD Notebook:
## Evaluation of fine-tuning the pre-trained SurvivEHR-CR model on a supervised cohort study.

Cohort study: predicting Cardiovascular Disease in a Type 2 Diabetes Mellitus population.

This notebook quantifies the performance obtained when fine-tuning the pre-trained model to a sub-population.

In [1]:
import os
from pathlib import Path
import sys
node_type = os.getenv('BB_CPU')
venv_dir = f'/rds/homes/g/gaddcz/Projects/CPRD/virtual-envTorch2.0-{node_type}'
venv_site_pkgs = Path(venv_dir) / 'lib' / f'python{sys.version_info.major}.{sys.version_info.minor}' / 'site-packages'
if venv_site_pkgs.exists():
    sys.path.insert(0, str(venv_site_pkgs))
    print(f"Added path '{venv_site_pkgs}' at start of search paths.")
else:
    print(f"Path '{venv_site_pkgs}' not found. Check that it exists and/or that it exists for node-type '{node_type}'.")

%load_ext autoreload
%autoreload 2

print(os.getcwd())

Added path '/rds/homes/g/gaddcz/Projects/CPRD/virtual-envTorch2.0-icelake/lib/python3.10/site-packages' at start of search paths.
/rds/homes/g/gaddcz/Projects/CPRD/examples/modelling/SurvStreamGPT/notebooks/CompetingRisk/CVD


In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import logging
import wandb
from tqdm import tqdm
import pickle
from hydra import compose, initialize
from omegaconf import OmegaConf
from CPRD.examples.modelling.SurvStreamGPT.run_experiment import run
from CPRD.data.foundational_loader import FoundationalDataModule
from CPRD.src.models.survival.task_heads.causal import SurvStreamGPTForCausalModelling

import time
import pyarrow.dataset as ds
import pyarrow.parquet as pq
import os
import polars as pl
pl.Config.set_tbl_rows(10000)
import pandas as pd
pd.options.display.max_rows = 10000

torch.manual_seed(1337)
torch.set_float32_matmul_precision('medium')

logging.basicConfig(level=logging.INFO)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = "cpu"    # if more informative debugging statements are needed
print(f"Using device: {device}.")

 # TODO: define an env variable to fix for a local hpc environment issue, this shouldn't be needed
%env SLURM_NTASKS_PER_NODE=28   

Using device: cuda.
env: SLURM_NTASKS_PER_NODE=28


# Choosing configurations
The default configuration is for pre-training. Here we modify as necesssary

Here we choose to load in the configuration for a small **pre-trained** 11.4M parameter model, named "CR_11M". We specfiy the `fine-tune` experiment type, which will lead to running the ```SupervisedExperiment```. 

We tell this experiment that we want to perform training (true by default). Additionally, we do choose to perform testing (true by default). As this is a supervised model, this tests the ability to predict the outcomes of interest. In this notebook, this is chosen to be those of the cohort study for predicting Cardiovascular Disease in a Type 2 Diabetes Mellitus population, and we add the folder containing this dataset to the configuration. 

```Note: As this is a supervised dataset, we need to tell the DataModule that the last event observed is a target and must be stripped. This is done by passing a list of targets to the configuration, overriding the null default. This lets the DataModule know that it should process batches as supervised.```

We set the number of workers to be appropriate for the number of CPUs available to reduce bottlenecking, and tell the experiment that we do not want to limit the number of testing batches. In addition, we specify where we want any checkpoints to be saved to avoid bloating the repository.

We design a new optimisation strategy for fine-tuning. Pre-training was achieved with a warmup and cosine annealing, with rates which are no appropriate for much smaller dataset sizes seen in clinical prediction models (CPMs). We here choose a simpler strategy: of ReduceOnPlateau with no warmup, increasing the number of epochs (default is 1) and reduced validation intervals, and the addition of early stopping. Additionally, as this is not a causal model we can increase the batch size. Finally, as this CPM is not trying to predict the value of any outcomes, we set the value weight to zero allowing the model to focus entirely on optimising survival outcome prediction.

# Small 11.4M parameter model

```
Hypertension
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃             Test metric             ┃            DataLoader 0             ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  Test:OutcomePerformanceMetricsctd  │         0.6980466246604919          │
│  Test:OutcomePerformanceMetricsibs  │         0.09058867272886643         │
│ Test:OutcomePerformanceMetricsinbll │         0.3116109046404026          │
│              test_loss              │         1.3693411350250244          │
└─────────────────────────────────────┴─────────────────────────────────────┘
CVD (old)
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃             Test metric             ┃            DataLoader 0             ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  Test:OutcomePerformanceMetricsctd  │         0.6582537293434143          │
│  Test:OutcomePerformanceMetricsibs  │        0.033626483186099766         │
│ Test:OutcomePerformanceMetricsinbll │         0.14357818411009718         │
│              test_loss              │          9.273333549499512          │
│          test_loss_desurv           │          9.273333549499512          │
└─────────────────────────────────────┴─────────────────────────────────────┘
```

In [9]:
pre_trained_models = ["CR_11M_24_11_01_big_posencscale_", "NULL"]
experiments = ["hypertension", "cvd"] 

for pre_trained_model in pre_trained_models:

    for experiment in experiments:
    
        # load the configuration file, override any settings 
        with initialize(version_base=None, config_path="../../../confs", job_name="testing_notebook"):
            cfg = compose(config_name="config_CompetingRisk37M", 
                          overrides=[# Experiment setup
                                     "experiment.type='few-shot'",
                                     f"experiment.run_id='{pre_trained_model}'",
                                     f"experiment.fine_tune_id='{experiment}-few-shot-time_test'",
                                     "experiment.train=True",
                                     "experiment.test=True",
                                     # Dataloader
                                     "data.meta_information_path=/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/PreTrain/meta_information_QuantJenny.pickle",
                                     "data.min_workers=12",
                                     # Optimiser
                                     "optim.num_epochs=5",
                                     "optim.scheduler=ReduceOnPlateau",
                                     "optim.scheduler_warmup=False",
                                     "optim.learning_rate=1e-3", #3e-4",
                                     "optim.val_check_interval=0.1",
                                     "optim.limit_val_batches=0.25",
                                     "optim.limit_test_batches=null",
                                     "optim.early_stop=True",
                                     "optim.early_stop_patience=1",
                                     # Head
                                     "head.surv_weight=1",
                                     "head.value_weight=0",
                                    ]
                         )     
    
        
        match experiment.lower():
            case "cvd":
                cfg.data.path_to_ds="/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_CVD/"
                cfg.experiment.fine_tune_outcomes=["IHDINCLUDINGMI_OPTIMALV2", "ISCHAEMICSTROKE_V2", "MINFARCTION", "STROKEUNSPECIFIED_V2", "STROKE_HAEMRGIC"]
            case "hypertension":
                cfg.data.path_to_ds="/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/"
                cfg.experiment.fine_tune_outcomes=["HYPERTENSION"]
                
        wandb.finish()
        experiment, dm = run(cfg)
        print(f"Loaded model with {sum(p.numel() for p in experiment.model.parameters())/1e6} M parameters")
        wandb.finish()


Scheduler,▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▁▁▁▁▁
train_loss,█▇██▇▇▇▆▁
trainer/global_step,▁▁▂▂▃▃▄▄▅▅▅▅▆▆▇▇██
Scheduler,0.001
epoch,0
train_loss,-3.58969
trainer/global_step,179


INFO:root:Running cr on 72 CPUs and 1 GPUs
INFO:root:====================================================================================================
INFO:root:# Loading DataModule for dataset /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/. This will be loaded in supervised form.
INFO:root:====================================================================================================
INFO:root:Creating supervised collator for DataModule
INFO:root:Using meta information from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/PreTrain/meta_information_QuantJenny.pickle
INFO:root:Using train file-row count dictionary from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/file_row_count_dict_train.pickle
INFO:root:Using test file-row count dictionary from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hyperte

/rds/bear-apps/2022a/EL8-ice/software/PyTorch-Lightning/2.1.0-foss-2022a-CUDA-11.7.0/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:630: Checkpoint directory /rds/projects/s/subramaa-mum-predict/CharlesGadd_Oxford/FoundationModelOutput/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:root:Using ReduceLROnPlateau scheduler
INFO:root:Not using warm-up in scheduler

  | Name         | Type                            | Params
-----------------------------------------------------------------
0 | model        | SurvStreamGPTForCausalModelling | 129 M 
  | other params | n/a                             | 1     
-----------------------------------------------------------------
1         Trainable params
129 M     Non-trainable params
129 M     Total params
517.425   Total estimated model params size (MB)
SLURM auto-requeueing enabled. Setting signal handlers.


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

INFO:root:scaling by Parameter containing:
tensor(-1.3863, device='cuda:0', requires_grad=True)
INFO:root:scaling by Parameter containing:
tensor(-1.3863, device='cuda:0', requires_grad=True)
INFO:root:scaling by Parameter containing:
tensor(-1.3863, device='cuda:0', requires_grad=True)
INFO:root:scaling by Parameter containing:
tensor(-1.3863, device='cuda:0', requires_grad=True)


...

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:root:scaling by Parameter containing:
tensor(-2.3186, device='cuda:0', requires_grad=True)
INFO:root:scaling by Parameter containing:
tensor(-2.3186, device='cuda:0', requires_grad=True)
INFO:root:scaling by Parameter containing:
tensor(-2.3186, device='cuda:0', requires_grad=True)
INFO:root:scaling by Parameter containing:
tensor(-2.3186, device='cuda:0', requires_grad=True)
INFO:root:scaling by Parameter containing:
tensor(-2.3186, device='cuda:0', requires_grad=True)
INFO:root:scaling by Parameter containing:
tensor(-2.3186, device='cuda:0', requires_grad=True)
INFO:root:scaling by Parameter containing:
tensor(-2.3186, device='cuda:0', requires_grad=True)
INFO:root:scaling by Parameter containing:
tensor(-2.3186, device='cuda:0', requires_grad=True)
INFO:root:scaling by Parameter containing:
tensor(-2.3186, device='cuda:0', requires_grad=True)
INFO:root:scaling by Parameter containing:
tensor(-2.3186, device='cuda:0', requires_grad=True)
INFO:root:scaling by Parameter containin

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:root:scaling by Parameter containing:
tensor(-3.2730, device='cuda:0', requires_grad=True)
INFO:root:scaling by Parameter containing:
tensor(-3.2730, device='cuda:0', requires_grad=True)
INFO:root:scaling by Parameter containing:
tensor(-3.2730, device='cuda:0', requires_grad=True)
INFO:root:scaling by Parameter containing:
tensor(-3.2730, device='cuda:0', requires_grad=True)
INFO:root:scaling by Parameter containing:
tensor(-3.2730, device='cuda:0', requires_grad=True)
INFO:root:scaling by Parameter containing:
tensor(-3.2730, device='cuda:0', requires_grad=True)
INFO:root:scaling by Parameter containing:
tensor(-3.2730, device='cuda:0', requires_grad=True)
INFO:root:scaling by Parameter containing:
tensor(-3.2730, device='cuda:0', requires_grad=True)
INFO:root:scaling by Parameter containing:
tensor(-3.2730, device='cuda:0', requires_grad=True)
INFO:root:scaling by Parameter containing:
tensor(-3.2730, device='cuda:0', requires_grad=True)
INFO:root:scaling by Parameter containin

SystemError: <built-in function _error_if_any_worker_fails> returned a result with an exception set

Subsets of data

In [6]:
pre_trained_models = ["CR_11M_24_11_01_big_posencscale_", "NULL"]
experiments = ["hypertension", "cvd"] 

for pre_trained_model in pre_trained_models:

    for experiment in experiments:
    
        # load the configuration file, override any settings 
        with initialize(version_base=None, config_path="../../../confs", job_name="testing_notebook"):
            cfg = compose(config_name="config_CompetingRisk37M", 
                          overrides=[# Experiment setup
                                     "experiment.type='few-shot'",
                                     f"experiment.run_id='{pre_trained_model}'",
                                     f"experiment.fine_tune_id='{experiment}-few-shot-cr3'",
                                     "experiment.train=True",
                                     "experiment.test=True",
                                     # Dataloader
                                     "data.meta_information_path=/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/PreTrain/meta_information_QuantJenny.pickle",
                                     "data.min_workers=12",
                                     "data.subsample_training=600",
                                     # Optimiser
                                     "optim.num_epochs=50",
                                     "optim.limit_test_batches=null",
                                     "optim.scheduler=ReduceOnPlateau",
                                     "optim.scheduler_warmup=False",
                                     # "optim.scheduler_periods=250",
                                     "optim.learning_rate=1e-3",
                                     "optim.val_check_interval=5",
                                     "optim.early_stop=True",
                                     "optim.early_stop_patience=20",
                                     # Head
                                     "head.surv_weight=1",
                                     "head.value_weight=0",
                                    ]
                         )     
    
        
        match experiment.lower():
            case "cvd":
                cfg.data.path_to_ds="/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_CVD/"
                cfg.experiment.fine_tune_outcomes=["IHDINCLUDINGMI_OPTIMALV2", "ISCHAEMICSTROKE_V2", "MINFARCTION", "STROKEUNSPECIFIED_V2", "STROKE_HAEMRGIC"]
            case "hypertension":
                cfg.data.path_to_ds="/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/"
                cfg.experiment.fine_tune_outcomes=["HYPERTENSION"]
                
        wandb.finish()
        experiment, dm = run(cfg)
        print(f"Loaded model with {sum(p.numel() for p in experiment.model.parameters())/1e6} M parameters")
        wandb.finish()


Scheduler,▁▁▁▁▁▁▁▁▁▁▁
Val:OutcomePerformanceMetricsctd,▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▆▆▆▆▆▇▇▇██▇▇█
Val:OutcomePerformanceMetricsibs,████▇▇▇▅▅▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▂▂▁▁▁▁▁▁
Val:OutcomePerformanceMetricsinbll,█▇▆▅▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train_loss,█▄▃▃▂▃▂▃▁▂▂
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
val_loss,█▆▅▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Scheduler,0.001
Val:OutcomePerformanceMetricsctd,0.48775
Val:OutcomePerformanceMetricsibs,0.08782


INFO:root:Running cr on 72 CPUs and 1 GPUs
INFO:root:====================================================================================================
INFO:root:# Loading DataModule for dataset /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/. This will be loaded in supervised form.
INFO:root:====================================================================================================
INFO:root:Creating supervised collator for DataModule
INFO:root:Using meta information from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/PreTrain/meta_information_QuantJenny.pickle
INFO:root:Using train file-row count dictionary from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hypertension/file_row_count_dict_train.pickle
INFO:root:Using test file-row count dictionary from /rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_Hyperte

/rds/bear-apps/2022a/EL8-ice/software/PyTorch-Lightning/2.1.0-foss-2022a-CUDA-11.7.0/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:630: Checkpoint directory /rds/projects/s/subramaa-mum-predict/CharlesGadd_Oxford/FoundationModelOutput/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:root:Using ReduceLROnPlateau scheduler
INFO:root:Not using warm-up in scheduler

  | Name  | Type                            | Params
----------------------------------------------------------
0 | model | SurvStreamGPTForCausalModelling | 129 M 
----------------------------------------------------------
541 K     Trainable params
128 M     Non-trainable params
129 M     Total params
517.425   Total estimated model params size (MB)
SLURM auto-requeueing enabled. Setting signal handlers.


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/rds/bear-apps/2022a/EL8-ice/software/PyTorch-Lightning/2.1.0-foss-2022a-CUDA-11.7.0/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:293: The number of training batches (10) is smaller than the logging interval Trainer(log_every_n_steps=20). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


...

Metric val_loss improved by 0.010 >= min_delta = 0. New best score: 2.756
Epoch 23, global step 235: 'val_loss' reached 2.75600 (best 2.75600), saving model to '/rds/projects/s/subramaa-mum-predict/CharlesGadd_Oxford/FoundationModelOutput/checkpoints/CR_11M_24_11_01_big_posencscale__hypertension-few-shot-cr3.ckpt' as top 1


Validation: |          | 0/? [00:00<?, ?it/s]

Metric val_loss improved by 0.008 >= min_delta = 0. New best score: 2.748
Epoch 23, global step 240: 'val_loss' reached 2.74786 (best 2.74786), saving model to '/rds/projects/s/subramaa-mum-predict/CharlesGadd_Oxford/FoundationModelOutput/checkpoints/CR_11M_24_11_01_big_posencscale__hypertension-few-shot-cr3.ckpt' as top 1


Validation: |          | 0/? [00:00<?, ?it/s]

/rds/bear-apps/2022a/EL8-ice/software/PyTorch-Lightning/2.1.0-foss-2022a-CUDA-11.7.0/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
INFO:root:Re-loading from best cached checkpoint /rds/projects/s/subramaa-mum-predict/CharlesGadd_Oxford/FoundationModelOutput/checkpoints/CR_11M_24_11_01_big_posencscale__hypertension-few-shot-cr3.ckpt

KeyboardInterrupt

wandb: Network error (ReadTimeout), entering retry loop.


In [ ]:


for batch in dm.train_dataloader():
    break

In [ ]:
# dm.train_set.view_sample(1, max_dynamic_events=4)

display(batch["tokens"][0, :])
display(batch["target_token"][0])

experiment.forward(batch)

In [ ]:
dm.encode(cfg.experiment.fine_tune_outcomes)
display(dm.decode([0]))
display(len(list(dm.train_set.tokenizer._event_counts["EVENT"])))

In [ ]:
# import wandb
print(model)
# wandb.finish()

In [ ]:
dm.encode(['IHDINCLUDINGMI_OPTIMALV2', 'ISCHAEMICSTROKE_V2', 'MINFARCTION', 'STROKEUNSPECIFIED_V2', 'STROKE_HAEMRGIC'])
# display(dm.decode([95, 175, 263,249]).split(" "))

In [ ]:
dm.tokenizer._event_counts["EVENT"][-5:].to_list()

In [ ]:
save_path = f"/rds/projects/s/subramaa-mum-predict/CharlesGadd_Oxford/FoundationModelOutput/checkpoints/{cfg.experiment.run_id}/"


# Load Pre-Trained model

In [ ]:
ckpt_path = cfg.experiment.log_dir + f'checkpoints/{cfg.experiment.run_id}.ckpt'
model = SurvivalExperiment.load_from_checkpoint(ckpt_path)

# Initialise fine-tuning data module

In [ ]:
# Update dataset path to point to the new dataset 
cfg.data.path_to_ds = "/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_CVD/"

# Build 
dm = FoundationalDataModule(path_to_db=cfg.data.path_to_db,
                            path_to_ds=cfg.data.path_to_ds,
                            load=True,
                            tokenizer="tabular",
                            batch_size=cfg.data.batch_size,
                            max_seq_length=cfg.transformer.block_size,
                            freq_threshold=cfg.data.unk_freq_threshold,
                            min_workers=cfg.data.min_workers,
                            overwrite_meta_information=cfg.data.meta_information_path,
                           )

vocab_size = dm.train_set.tokenizer.vocab_size
print(f"{vocab_size} vocab elements")

# list of univariate measurements to model with Normal distribution
# Extract the measurements, using the fact that the diagnoses are all up upper case.
measurements_for_univariate_regression = [record for record in dm.tokenizer._event_counts["EVENT"] if record.upper() != record]
cfg.head.tokens_for_univariate_regression = dm.encode(measurements_for_univariate_regression) 
# display(measurements_for_univariate_regression)

In [ ]:
# # import pickle as pkl
# # import pathlib

# pkl_file_to_amend = "/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_CVD/file_row_count_dict_test.pickle"

# with open(pkl_file_to_amend, 'rb') as pickle_file:
#     content = pickle.load(pickle_file)
# display(content)

# # new_dictionary = {}
# # for key in content.keys():
# #     str_to_remove = "/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/FoundationalModel/FineTune_CVD/split=val/"
# #     new_key = str(key)[len(str_to_remove):]
# #     new_dictionary[new_key] = content[key]
# # display(new_dictionary)


# # with open(pkl_file_to_amend, 'wb') as handle:
# #     pickle.dump(new_dictionary, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
# new_dictionary

In [ ]:
import copy
start = time.time()   # starting time
for batch in dm.train_dataloader():
    # print(batch["tokens"][1,:])
    
    c_batch = convert_batch_to_none_causal(batch)
    # print(c_batch["tokens"][1,:])
    # print(c_batch["target_token"][1])

    # print(batch["tokens"][1,:])
    
    break
    
print(f"batch loaded in {time.time()-start} seconds")    
    
# for key in batch.keys():
#     print(f"{key}".ljust(20) + f"{batch[key].shape}")

# tokens = batch["tokens"][0].tolist()    
# sentence = dm.decode(tokens).split(" ")
# for token, value in zip(sentence, batch["values"][0].tolist()):
#     print(f"{token}:".ljust(40) + f"{value}")

In [ ]:
display(batch.keys())
display(c_batch.keys())

print(batch["static_covariates"].shape)

# print(dm.train_set.static_1hot)
# print(dm.train_set.static_1hot["SEX"].categories_)
# print(dm.train_set.static_1hot["IMD"].categories_)
# print(dm.train_set.static_1hot["ETHNICITY"].categories_)

print(batch["tokens"][1,:])
print(c_batch["tokens"][1,:])
print(c_batch["target_token"][1])

## View an example sample

In [ ]:
dm.test_set.view_sample(11003, max_dynamic_events=None, report_time=True)

# Custom wrapper prediction last token

To begin with, I will just loop over samples individually to test the zero-shot capacity of SurvivEHR. 

In [ ]:


# Verifying on datamodule 
for _idx, batch in enumerate(dm.test_dataloader()):
    if _idx > 10:
        break
    print(_idx)
    print(torch.stack([batch["tokens"][10,:5], 
                       batch["values"][10,:5],  
                       batch["ages"][10,:5],
                       batch["attention_mask"][10,:5]]))
    batch = replace_last_non_pad_with_pad(batch)
    print(torch.stack([batch["tokens"][10,:5], 
                       batch["values"][10,:5],  
                       batch["ages"][10,:5],
                       batch["attention_mask"][10,:5]]))

In [ ]:
outcome_of_interest = ["COPD", "SUBSTANCEMISUSE"]
outcome_token = dm.encode(outcome_of_interest)[0]
print(outcome_token)
# print(model(batch))

In [ ]:
Hs, labels = [], []
mins,maxes=[],[]
for _idx, batch in enumerate(dm.test_dataloader()):

    batch = replace_last_non_pad_with_pad(batch)
    print(batch["tokens".shape)
    outputs, _, hidden_states = model(batch, is_generation=True)
    print(outputs)
    
    hidden_states = hidden_states.cpu().detach().numpy()                           # (64, 128, 384) 
    Hs.append( hidden_states.reshape(hidden_states.shape[0], -1) )
    labels.append((batch["target_token"] == outcome_token).long().numpy())

    if _idx == 9:
        break



# Visualise hidden dimension labelled by target

In [ ]:
import umap
from sklearn.preprocessing import StandardScaler

H = np.concatenate(Hs, 0)
lbl = np.concatenate(labels, 0)

H = StandardScaler().fit_transform(H)
reducer = umap.UMAP()
H_proj = reducer.fit_transform(H)

plt.close()
plt.scatter(H_proj[:,0], H_proj[:,1], c=lbl)
plt.savefig(save_path + f"zero_shot/hidden_umap.png")

In [ ]:
print(outputs["surv"]["surv_CDF"][outcome_token].shape)

# The first two tokens in the vocab correspond to the PAD and UNK tokens. There is no CDF corresponding to the PAD token, so the indexing for surv_CDF begins as ["UNK", "ADDISONS_DISEASE", ...]
# print(dm.decode([0,1,2]))

outcomes = ["COPD", "SUBSTANCEMISUSE"]
outcome_tokens = dm.encode(outcomes)

# for outcome in outcomes:
    # observed_outcome_token = dm.encode([outcome])[0]
cdf = np.zeros_like(outputs["surv"]["surv_CDF"][0])
lbls = np.zeros_like(batch["target_token"])

for _outcome_token in outcome_tokens:
    cdf += outputs["surv"]["surv_CDF"][_outcome_token - 1] 
    lbls += (batch["target_token"] == _outcome_token).long().numpy()

plt.close()
cdf_true = cdf[lbls==1,:]
cdf_false = cdf[lbls==0,:]
for i in range(cdf_true.shape[0]):
    plt.plot(np.linspace(1,1826,1826), cdf_true[i,:], c="r", label="outcome occurred next" if i == 0 else None, alpha=1)
for i in range(cdf_false.shape[0]):
    plt.plot(np.linspace(1,1826,1826), cdf_false[i,:], c="k", label="outcome did not occur next" if i == 0 else None, alpha=0.3)

plt.legend(loc=2)
plt.xlabel("days")
plt.ylabel(f"P(t>T) - outcomes={','.join(outcomes)}")
plt.savefig(save_path + f"zero_shot/cdf_outcomes.png")

In [ ]:
print(batch["target_token"].unique())
print(len(outputs["surv"]["surv_CDF"]))

In [ ]:
dm.decode([2])

In [ ]:
outputs["surv"]["surv_CDF"][observed_outcome_token - 1]